In [1]:
import warnings
import time as tm
import os
import datetime
from collections import namedtuple
import os.path
from collections import namedtuple
from os.path import normpath, realpath
from os.path import join as pathjoin
from datetime import datetime
from time import time
import csv
import pandas as pd

In [2]:
rootdir=['/mnt/docker','/mnt/lesson','/mnt/picture','/mnt/private','/mnt/software','/mnt/usbshare1-2','/mnt/video']
outfile="/home/hadoopusr/sambashare/dir.csv"
warnings.filterwarnings("ignore")

In [3]:
os.remove(outfile)
df = pd.DataFrame(list())
df.to_csv(outfile)

In [4]:
_wrap_entry = namedtuple( 'DirEntryWrapper', 'name path islink size' )
def scantree( rootdir, follow_links=False, reldir='' ):
    visited = set()
    rootdir = normpath(rootdir)
    with os.scandir(rootdir) as it:
        for entry in it:
            if entry.is_dir():
                if not entry.is_symlink() or follow_links:
                    absdir = realpath(entry.path)
                    if absdir in visited: 
                        continue 
                    else: 
                        visited.add(absdir)
                    yield from scantree( entry.path, follow_links, pathjoin(reldir,entry.name) )
            else:
                yield _wrap_entry( 
                    pathjoin(reldir,entry.name), 
                    entry.path, 
                    entry.is_symlink(),
                    entry.stat().st_size )

In [5]:
def creation_date(filename):
    t = os.path.getmtime(filename)
    return datetime.fromtimestamp(t)

In [6]:
start_time = tm.time()
# df2 = pd.DataFrame([[2,3,4]])
# pd.concat([df2, df])
all_results = []  # <-- list for all results
for path in rootdir:
    for entry in scantree(path, follow_links=False):
        fname=entry.name
        filepath = entry.path 
        #crtdt=datetime.fromtimestamp(creation_date(filepath)).strftime("%d/%m/%Y, %H:%M:%S")
        crtdt=creation_date(filepath).strftime("%d/%m/%Y--%H:%M:%S")
        filesize = str(round(entry.size/(1024*1024)))
        type=entry.name.split(".")[-1]
        #print(type(c1))
        #print(crtdt)
        #filesize = str(round(entry.size/(1024*1024)))
        #print(filepath+":"+filesize+":"crtdt)
        #print(fname+":"+filepath+":"+filesize+":"+crtdt+":"+type)
        #df2 = pd.DataFrame([[fname,filepath,crtdt,filesize,type]])
        #pd.concat([df, df2])
        
        new_data = fname+','+filepath+','+crtdt+','+filesize+','+type
        #new_line='\n'.join(new_data)
        #all_results.append(new_data)  # <-- keep result 
        all_results.append(new_data)  # <-- keep result 
        # with open('/home/hadoopusr/sambashare/dir.csv', 'a', newline='') as file:
        #     writer = csv.writer(file)
        #     writer.writerows(new_data)
        #print(type(new_data))
        #df_new = spark.createDataFrame(new_data,['name','dir','crt_time','fsize','ftype'])
        #df_new.show()
        #df_new.write.insertInto('dc.nas', overwrite=False)
        #df.append({fname,filepath,crtdt,filesize,type},ignore_index=True)
    
    #df.head(5)
    
    # # r=root, d=directories, f = files
    # for r, d, f in os.walk(thisdir):
    #     for file in f:
    #         if file.endswith(".sh"):
    #             path=os.path.join(r, file)
    #             size = str(os.stat(path).st_size)
    #             print(path+"Size:"+size)
    #     # for dlist in d:
    #     #     print(dlist)
    #write_file = "/home/hadoopusr/sambashare/dir.csv"
with open(outfile, "wt", encoding="utf-8") as output:
    for line in all_results:
        output.write(line + '\n')
print("Whole loading takes --- %s seconds ---" % (tm.time() - start_time))

Whole loading takes --- 1457.4541935920715 seconds ---
